# Avaliação de Modelos de Geração de Imagens

A avaliação de modelos generativos, especificamente aqueles baseados em difusão, apresenta desafios únicos quando comparada a tarefas supervisionadas clássicas. Não existe uma métrica única que capture toda a complexidade da geração de imagens; portanto, é necessário analisar o desempenho sob múltiplas óticas: qualidade perceptual (fidelidade), diversidade da distribuição gerada e alinhamento semântico com o prompt de entrada.

In [ ]:
import torch
import numpy as np
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.inception import InceptionScore
from torchmetrics.multimodal.clip_score import CLIPScore
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

In [ ]:
# Definição do dispositivo de processamento
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device being used: {device}")

## Geração com Stable Diffusion 1.5

O Stable Diffusion 1.5 é um modelo de difusão latente (LDM) que opera no espaço comprimido de um autoencoder variacional (VAE), em vez de operar diretamente no espaço de pixel. Isso reduz drasticamente o custo computacional. Para a inferência, utilizamos a biblioteca `diffusers`, que abstrai o pipeline de denoising, o agendador (scheduler) e a decodificação via VAE. A tarefa de *text-to-image* envolve condicionar o processo de denoising reverso através de embeddings de texto gerados por um codificador CLIP.

In [ ]:
# Carregamento do pipeline pré-treinado
model_id = "runwayml/stable-diffusion-v1-5"
pipe_txt2img = StableDiffusionPipeline.from_pretrained(model_id).to(device)

In [ ]:
prompt = "a photograph of an astronaut riding a horse on mars, high resolution, 8k"
negative_prompt = "blurry, low quality, distorted"

image_generated = pipe_txt2img(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=50,
    guidance_scale=7.5
).images[0]

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(image_generated)
plt.axis("off")
plt.title("Generated Image (Text-to-Image)")
plt.show()

### Pipeline Image-to-Image

A tarefa de *image-to-image* difere da geração pura pois o processo de difusão não inicia a partir de um ruído gaussiano puro $\mathcal{N}(0, I)$. Em vez disso, a imagem de entrada é codificada para o espaço latente e ruído é adicionado a ela até um certo passo de tempo $t$, controlado por um hiperparâmetro frequentemente denominado *strength*. O processo de denoising reverso então guia a geração em direção ao novo prompt, mantendo a estrutura semântica e composicional da imagem original.

In [ ]:
# Reutilizamos os componentes do pipe anterior para economizar memória
pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None
)
pipe_img2img = pipe_img2img.to(device)

In [ ]:
prompt_i2i = "a oil painting of an astronaut riding a horse on mars, van gogh style"

# Geração a partir da imagem anterior
image_i2i = pipe_img2img(
    prompt=prompt_i2i,
    image=image_generated,
    strength=0.75,
    guidance_scale=8.0
).images[0]

plt.figure(figsize=(8, 8))
plt.imshow(image_i2i)
plt.axis("off")
plt.title("Generated Image (Image-to-Image)")
plt.show()

In [ ]:
import requests

url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
response = requests.get(url)
init_image = Image.open(BytesIO(response.content)).convert("RGB")
init_image = init_image.resize((768, 512)) # Redimensionando para tamanho padrão

plt.figure(figsize=(8, 8))
plt.imshow(init_image)
plt.axis("off")
plt.title("Initial Image")
plt.show()

In [ ]:
prompt_i2i = "a fantasy landscape, realistic mountains, 8k, photorealistic, lord of the rings style"

image_i2i = pipe_img2img(
    prompt=prompt_i2i,
    image=init_image,
    strength=0.75,
    guidance_scale=8.0
).images[0]

plt.figure(figsize=(8, 8))
plt.imshow(image_i2i)
plt.axis("off")
plt.title("Generated Image (Image-to-Image)")
plt.show()

## Fréchet Inception Distance (FID)

A Fréchet Inception Distance (FID) é a métrica padrão-ouro para avaliar o realismo e a diversidade de imagens geradas. Ela calcula a distância de Wasserstein-2 entre duas distribuições multivariadas de Gaussianas ajustadas às características extraídas de uma rede Inception-v3 pré-treinada (geralmente na camada de *pooling* anterior à classificação).

Sejam $(\mu_r, \Sigma_r)$ a média e a matriz de covariância das características das imagens reais, e $(\mu_g, \Sigma_g)$ as estatísticas das imagens geradas. O FID é definido como:

$$
FID = ||\mu_r - \mu_g||^2 + Tr(\Sigma_r + \Sigma_g - 2(\Sigma_r \Sigma_g)^{1/2})
$$

Um valor de FID menor indica que a distribuição das imagens geradas é mais próxima da distribuição das imagens reais, implicando melhor qualidade visual e diversidade similar.

In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

to_uint8_299 = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    lambda x: (x * 255).to(torch.uint8)
])

prompts_batch = [
    "a photograph of an astronaut riding a horse on mars, high resolution",
    "a cyberpunk city at night, ultra detailed, 8k",
    "a portrait of a medieval knight, cinematic lighting",
    "a fantasy landscape with dragons and castles, 8k",
    "a close up photo of a cat wearing sunglasses, 4k",
]

negative_prompts_batch = ["blurry, low quality, distorted"] * len(prompts_batch)

with torch.no_grad():
    out = pipe_txt2img(
        prompt=prompts_batch,
        negative_prompt=negative_prompts_batch,
        num_inference_steps=30,
        guidance_scale=7.5
    )

generated_images_uint8 = torch.stack(
    [to_uint8_299(img) for img in out.images]
).to(device)

print("Shape das imagens geradas:", generated_images_uint8.shape)

In [ ]:
fig, axes = plt.subplots(1, len(prompts_batch), figsize=(3 * len(prompts_batch), 3))
for i, ax in enumerate(axes):
    ax.imshow(generated_images_uint8[i].cpu().permute(1, 2, 0).numpy())
    ax.set_title(f"Sample {i}")
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# Dataset real para servir de referência nas métricas (exemplo com CIFAR-10)
real_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    lambda x: (x * 255).to(torch.uint8)
])

real_dataset = datasets.CIFAR10(
    root="data/data_cifar",
    train=True,
    download=True,
    transform=real_transform
)

real_loader = DataLoader(real_dataset, batch_size=len(prompts_batch), shuffle=True)
real_images_uint8, _ = next(iter(real_loader))
real_images_uint8 = real_images_uint8.to(device)

print("Shape das imagens reais para as métricas:", real_images_uint8.shape)

In [ ]:
# Inicialização da métrica
fid = FrechetInceptionDistance(feature=64).to(device)

# Atualiza com imagens reais
fid.update(real_images_uint8, real=True)

# Atualiza com imagens geradas pelo Stable Diffusion
fid.update(generated_images_uint8, real=False)

# Computação do score
fid_score = fid.compute()
print(f"FID Score: {fid_score.item()}")

In [ ]:
fid.reset()

# Simulação de dados para exemplo
fake_images = torch.randint(0, 255, (5, 3, 299, 299), dtype=torch.uint8).to(device)

fid.update(real_images_uint8, real=True)
fid.update(fake_images, real=False)

fid_score = fid.compute()
print(f"FID Score: {fid_score.item()}")

## Inception Score (IS)

O Inception Score (IS) avalia a qualidade das imagens focando em dois aspectos: nitidez (se a imagem é claramente identificável como uma classe específica) e diversidade (se o modelo gera uma variedade de classes). Ele utiliza a divergência KL entre a distribuição condicional de classes $p(y|x)$ e a distribuição marginal $p(y)$.

Matematicamente, para um conjunto de imagens geradas $x$, o IS é calculado como:

$$
IS = \exp\left( \mathbb{E}_{x \sim p_g} [ D_{KL}(p(y|x) || p(y)) ] \right)
$$

Onde $p(y|x)$ é a probabilidade da classe dada a imagem (calculada pela Inception-v3) e $p(y) \approx \frac{1}{N} \sum_{i=1}^N p(y|x_i)$ é a distribuição marginal das classes. Valores maiores indicam melhor desempenho.

In [ ]:
# Inicialização da métrica
inception_score = InceptionScore(feature="logits_unbiased").to(device)

# Aqui usamos o mesmo batch generated_images_uint8
inception_score.update(generated_images_uint8)

is_mean, is_std = inception_score.compute()

print(f"Inception Score: Mean = {is_mean.item()}, Std = {is_std.item()}")

In [ ]:
imgs_generated_batch = torch.randint(0, 255, (32, 3, 299, 299), dtype=torch.uint8).to(device)

# Atualização e cálculo
inception_score.update(imgs_generated_batch)
is_mean, is_std = inception_score.compute()

print(f"Inception Score: Mean = {is_mean.item()}, Std = {is_std.item()}")

## CLIP Score

Diferente das métricas anteriores que avaliam a distribuição da imagem, o CLIP Score mede o alinhamento semântico entre o prompt de texto e a imagem gerada. Ele utiliza o modelo CLIP (Contrastive Language-Image Pre-training) que projeta texto e imagem em um espaço latente compartilhado.

O CLIP Score é definido como a similaridade de cosseno entre o embedding da imagem $E_I$ e o embedding do texto $E_T$, frequentemente escalado por um fator:

$$
CLIP(I, T) = \max(100 \cdot \cos(E_I, E_T), 0)
$$

Essa métrica é crucial para verificar se o modelo está obedecendo às condicionantes textuais fornecidas pelo usuário.

In [ ]:
# Inicialização da métrica CLIP
metric_clip = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16").to(device)

# Normalizar para [0, 1] como float
images_for_clip = (generated_images_uint8.float() / 255.0).to(device)

clip_score = metric_clip(images_for_clip, prompts_batch)

print(f"CLIP Score: {clip_score.detach().item()}")